# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("cities.csv")

weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Dunmore Town,2,BS,1601257965,70,25.50,-76.65,83.01,7.99
1,1,Hermanus,0,ZA,1601257965,93,-34.42,19.23,37.99,5.73
2,2,Rikitea,0,PF,1601257966,78,-23.12,-134.97,74.64,11.10
3,3,Torbay,90,CA,1601257966,93,47.67,-52.73,64.00,21.92
4,4,Pevek,100,RU,1601257967,78,69.70,170.31,37.11,24.29
...,...,...,...,...,...,...,...,...,...,...
562,562,Sirte,27,LY,1601258152,47,31.21,16.59,81.43,5.84
563,563,Quesnel,90,CA,1601258152,66,53.00,-122.50,53.60,10.29
564,564,Ahipara,0,NZ,1601258153,54,-35.17,173.17,59.14,14.34
565,565,Wairoa,4,NZ,1601258153,38,-39.03,177.37,61.45,13.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]


In [4]:


# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

max_intensity = np.max(humidity)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig



max_intensity




100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
vacation_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
63,63,Albany,0,US,1601257953,94,42.60,-73.97,73.00,3.96
66,66,Trairi,0,BR,1601257991,83,-3.28,-39.27,77.36,9.62
122,122,Tiznit Province,0,MA,1601258009,53,29.58,-9.50,71.65,1.90
127,127,São Félix do Xingu,0,BR,1601258011,67,-6.64,-51.99,75.43,2.04
159,159,Katsuura,0,JP,1601258021,50,35.13,140.30,78.01,7.00
237,237,Marsá Maţrūḩ,0,EG,1601258046,83,31.35,27.25,73.40,7.63
239,239,Wenshang,0,CN,1601257770,1,35.73,116.50,77.00,6.62
443,443,Riyadh,0,SA,1601257957,24,24.69,46.72,78.80,3.36
513,513,Sakakah,0,SA,1601258136,22,29.97,40.21,78.80,6.93
520,520,Kiryat Gat,0,IL,1601258138,70,31.61,34.76,75.99,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = vacation_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Albany,US,42.60,-73.97,
66,Trairi,BR,-3.28,-39.27,
122,Tiznit Province,MA,29.58,-9.50,
127,São Félix do Xingu,BR,-6.64,-51.99,
159,Katsuura,JP,35.13,140.30,
237,Marsá Maţrūḩ,EG,31.35,27.25,
239,Wenshang,CN,35.73,116.50,
443,Riyadh,SA,24.69,46.72,
513,Sakakah,SA,29.97,40.21,
520,Kiryat Gat,IL,31.61,34.76,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # build url and make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
     # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 63: Albany.
Missing field/result... skipping.
Retrieving Results for Index 66: Trairi.
Closest hotel in Trairi is Rede Beach: Hotel em Guajiru.
Retrieving Results for Index 122: Tiznit Province.
Missing field/result... skipping.
Retrieving Results for Index 127: São Félix do Xingu.
Closest hotel in São Félix do Xingu is Apart Hotel Rio Xingu.
Retrieving Results for Index 159: Katsuura.
Closest hotel in Katsuura is Katsuura Hilltop Hotel & Residence.
Retrieving Results for Index 237: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Jewel Matrouh Hotel.
Retrieving Results for Index 239: Wenshang.
Closest hotel in Wenshang is Green Tree Inn Jining Wenshang Baoxiang Temple Express Hotel.
Retrieving Results for Index 443: Riyadh.
Closest hotel in Riyadh is Hyatt Regency Riyadh Olaya.
Retrieving Results for Index 513: Sakakah.
Closest hotel in Sakakah is Raoum Inn Hotel.
Retrieving Results for Index 520: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
Retrie

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Albany,US,42.60,-73.97,
66,Trairi,BR,-3.28,-39.27,Rede Beach: Hotel em Guajiru
122,Tiznit Province,MA,29.58,-9.50,
127,São Félix do Xingu,BR,-6.64,-51.99,Apart Hotel Rio Xingu
159,Katsuura,JP,35.13,140.30,Katsuura Hilltop Hotel & Residence
237,Marsá Maţrūḩ,EG,31.35,27.25,Jewel Matrouh Hotel
239,Wenshang,CN,35.73,116.50,Green Tree Inn Jining Wenshang Baoxiang Temple...
443,Riyadh,SA,24.69,46.72,Hyatt Regency Riyadh Olaya
513,Sakakah,SA,29.97,40.21,Raoum Inn Hotel
520,Kiryat Gat,IL,31.61,34.76,Desert Gat


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info =[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
 

# Add marker layer and info box content on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig



Figure(layout=FigureLayout(height='420px'))